In [1]:
import pandas as pd
import time
import numpy as np
import copy
import glob
import sys, os
import torch
import torch.nn as nn
import torch.utils.data as data_utils
import pickle
from sklearn import preprocessing
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from folktables import ACSDataSource, ACSEmployment,ACSIncome
import matplotlib.pyplot as plt
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.metrics import roc_curve
from sklearn.model_selection import StratifiedKFold, train_test_split
from torchvision import models
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
import os
from torch.utils.data import Dataset
import numpy as np
import torch.nn.functional as F
import pandas as pd
from scipy.stats import multivariate_normal
import torch, random, copy, os
from collections import OrderedDict


In [2]:
# !pip install ray

# data preparation

In [3]:
os.getcwd()

'/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB'

In [4]:
 
class DeepNet(nn.Module):
    def __init__(self):
        super().__init__()
        # 14 : input shape
        # 9-> we have 9 columns in data 
        self.layer1 = nn.Linear(9, 512)
        self.act1 = nn.ReLU()
        self.dropout1 = nn.Dropout(p=0.5)
        self.layer2 = nn.Linear(512, 256)
        self.act2 = nn.ReLU()
        self.layer3 = nn.Linear(256, 60)
        self.act3 = nn.ReLU()
        self.output = nn.Linear(60, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.act1(self.layer1(x))
        x = self.dropout1(x)
        x = self.act2(self.layer2(x))
        x = self.act3(self.layer3(x))
        x = self.sigmoid(self.output(x))
        return x


class Client:
    def __init__(self):
        self.client_id: int = None
        self.valset: DataLoader = None
        self.trainset: DataLoader = None
        self.testset: DataLoader = None
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
        self.model = DeepNet().to(self.device)
        self.criterion = torch.nn.BCELoss()

    def get_client_local_dataset(self):
        
        print("self.client_id:: ",self.client_id)

        temp_path_data="/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/client_df"
        
        with open(temp_path_data+"/clients_training.pkl", "rb") as f:
            self.trainset = pickle.load(f)
            
        with open(temp_path_data+"/clients_validation.pkl", "rb") as f:
            self.valset = pickle.load(f)
         
        with open(temp_path_data+"/clients_testing.pkl", "rb") as f:
            self.testset  = pickle.load(f)

        self.trainset = self.trainset[self.client_id]
        self.valset = self.valset[self.client_id]       
        self.testset = self.testset[self.client_id]

    def train(self, client_id: int, model_params: OrderedDict[str, torch.Tensor], num_epochs=5, learning_rate=0.001):
        self.client_id = client_id
        self.get_client_local_dataset()

        # Define loss function and optimizer
        criterion = nn.BCELoss()
        optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)

        for epoch in range(num_epochs):
            # Set the model to training mode
            self.model.train()

            for inputs, labels in self.trainset:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels.float())
                loss.backward()
                optimizer.step()

            # Calculate and print the training accuracy for this epoch (optional)
            correct = 0
            total = 0
            size = 0
            loss = 0
            correct = 0
            total = 0
            predicted_labels = []
            true_labels = []
            
            with torch.no_grad():
                for inputs, labels in self.valset:
                    inputs, labels = inputs.to(self.device), labels.to(self.device)
                    
                    outputs = self.model(inputs)
                    loss += self.criterion(outputs, labels)
                    predicted = outputs > 0.5

                    predicted_labels.extend(predicted.cpu().numpy())
                    true_labels.extend(labels.cpu().numpy())

                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

            loss = loss / len(self.valset)        
            accuracy = correct / total
            
            # accuracy = 100 * correct / total
            print(f"Epoch {epoch+1}/{num_epochs}, Training Accuracy: {accuracy:.2f}%")

        # Optionally, save the trained model parameters
        client_model_path = f"/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/models/client_{self.client_id}_model.pth"
        torch.save(self.model.state_dict(), client_model_path)

        # Return the trained model parameters
        return list(self.model.state_dict().values()), len(self.trainset.dataset)
        

In [7]:
class Serverbase:
    def __init__ (self,model):
        self.model = model
        self.num_rounds=2
        self.local_epoch=5
        self.optimizer=2
        self.lr=0.001
        self.beta=0.2
        self.batch_size=32
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

        self.client_id_indices, self.client_num_in_total = [0,1], 2        
        self.updated_params_cache = []
        self.weights_cache = []
        self.global_params_dict: OrderedDict[str : torch.Tensor] = None

        self.backbone=DeepNet
        _dummy_model = self.backbone()
        self.global_params_dict = OrderedDict(_dummy_model.state_dict())

        self.temp_dir="/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/models/"
        
    def fairFed(self, num_rounds = 2, local_epochs = 2, learning_rate = 0.005, beta = 0.3, alpha = 0.1, optimizer = 'adam'):
        
        for round_ in tqdm(range(num_rounds)):
  
            selected_clients=[0,1]
            
            for client_id in selected_clients:
                client = Client()

                updated_params, weight = client.train(client_id, self.model.state_dict(), num_epochs=local_epochs, learning_rate=learning_rate)

                # store it for SecAgg
                self.updated_params_cache.append(updated_params)
                self.weights_cache.append(weight)

            self.aggregate_parameters_FedAvg(self.updated_params_cache, self.weights_cache)
            torch.save(self.global_params_dict, os.path.join(self.temp_dir, "global_model.pt"),)
    

    @torch.no_grad()
    def aggregate_parameters_FedAvg(self, updated_params_cache, weights_cache):
        weight_sum = sum(weights_cache)
        weights = torch.tensor(weights_cache, device=self.device) / weight_sum

        aggregated_params = []

        for params in zip(*updated_params_cache):
            aggregated_params.append(
                torch.sum(weights * torch.stack(params, dim=-1), dim=-1)
            )

        self.global_params_dict = OrderedDict(
            zip(self.global_params_dict.keys(), aggregated_params)
        )

                                

In [9]:
model = DeepNet()
server = Serverbase(model)
server.fairFed()  

  0%|                                                                                                                        | 0/2 [00:00<?, ?it/s]

self.client_id::  0
Epoch 1/2, Training Accuracy: 0.72%
Epoch 2/2, Training Accuracy: 0.75%
self.client_id::  1
Epoch 1/2, Training Accuracy: 0.61%


 50%|████████████████████████████████████████████████████████                                                        | 1/2 [00:02<00:02,  2.30s/it]

Epoch 2/2, Training Accuracy: 0.70%
self.client_id::  0
Epoch 1/2, Training Accuracy: 0.73%
Epoch 2/2, Training Accuracy: 0.73%
self.client_id::  1


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:04<00:00,  2.15s/it]

Epoch 1/2, Training Accuracy: 0.61%
Epoch 2/2, Training Accuracy: 0.55%
